In [83]:
import polars as pl
from ergochemics.standardize import standardize_smiles
from functools import partial

In [84]:
std_smi = partial(standardize_smiles, neutralization_method="simple", quiet=True)

In [85]:
name_smi = {}
with open("/home/stef/bottle/artifacts/4228703247252781668.txt", "r") as f:
    for line in f.readlines():
        split_line = line.strip().split("\t")
        if len(split_line) == 2:
            name, smi = split_line
            name_smi[name] = std_smi(smi)
        

In [86]:
beachheads = [
    "D-Xylose",
    "Protocatechuic acid",
    "Pyruvate",
    "Malonyl-CoA",
    "L-aspartate",
    "Citrate",
    "Geranyl diphosphate",
    "Geranylgeranyl diphosphate",
    "Farnesyl diphosphate",
    "L-glutamate",
]

potential_beachheads = [
    "Glycerol",
    "L-Tyrosine",
    "Prephenic acid",
    "Chorismate",
    "Acetolactate",
    "2-Ketoisovalerate",
    "Acetoacetyl-CoA",
    "Acetyl-CoA",
    "2-Ketobutyric acid",
    "Propionyl-CoA",
    "L-lysine",
    "Glutaric acid",
    "Succinyl-CoA",
    "L-Proline",
    "L-Arginine",
]

targets = [
    "cis,cis-Muconic acid",
    "3-Hydroxypropionic acid",
    "cis-Aconitate",
    "Cineole",
    "Bisabolene",
    "Kaurene"
]

In [87]:
missing_smiles = []
for group_name, group in zip(["beachheads", "potential_beachheads", "targets"], [beachheads, potential_beachheads, targets]):
    for name in group:
        missing_smiles.append(name) if name not in name_smi else None

In [88]:
missing_smiles

[]

In [89]:
with open("round_2.txt", "w") as f:
    for name in missing_smiles:
        f.write(f"{name}\n")

In [90]:
table = []
for name, smi in name_smi.items():
    table.append(
        [name, smi, name in beachheads, name in potential_beachheads, name in targets]
    )   
    
df = pl.DataFrame(
    table,
    schema=[
        ("name", pl.String),
        ("smiles", pl.String),
        ("beachhead", pl.Boolean),
        ("potential_beachhead", pl.Boolean),
        ("target", pl.Boolean),
    ]
)
df.head()

/tmp/ipykernel_175240/18072210.py:7: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(


name,smiles,beachhead,potential_beachhead,target
str,str,bool,bool,bool
"""Glucose""","""OCC1OC(O)C(O)C(O)C1O""",false,false,false
"""Pyruvate""","""CC(=O)C(=O)O""",true,false,false
"""D-xylono-1,4-lactone""","""O=C1OC(CO)C(O)C1O""",false,false,false
"""4-Hydroxyhexan-3-one""","""CCC(=O)C(O)CC""",false,false,false
"""Propanal""","""CCC=O""",false,false,false


In [91]:
df.write_csv(
    "/home/stef/bottle/artifacts/agile_biofoundry.tsv",
    separator="\t",
)